In [26]:
import os
import json

parent = "/Users/finn/Documents/GitHub/FM-ds.github.io/E4E/work"

ensure_subtitle_space = True
update_descriptions = True

### Standardising Vega Parameters
The code in the cell below loops through all vega-lite specs in the parent path <b> and all sub-directories </b> and standardises the parameters listed in standardise_params. It preserve any parameters not specified in standardise_params that are not sub-parameters of a parameter specified in standardise_params (e.g. would preserve title in the example below). It <will not> preserve any sub-sub-parameters (e.g. would not preserve encoding.x.title if encoding.x.scale were specified in standard_params).

Additionally if ensure_subtitle_space, it ensure the final line of the subtitle is an empty line, adding one if needed. If update_descriptions is True, it will set the description of the spec to the subtitle, to ensure ARIA accesibility.

In [37]:

standard_params = {
    "title" : {
        "fontSize": 16,
        "subtitleFontSize": 12,
        "subtitleFontStyle": "italic",
    }
}

# NB: This preserves subparameters that are not in the standard_params dict (e.g. would preserve "subtitle" and "title")
# But will not preserve sub-sub-parameters

# Recursively walk through the directory and subdirectories
for root, dirs, files in os.walk(parent):
    for file in [file for file in files if file.endswith(".json")]: # Only look at json files
        spec = json.load(open(os.path.join(root, file), "r"))
        if '$schema' in spec and "vega.github.io/schema" in spec['$schema']: # Only look at vega files
            for param in standard_params:
                if param not in spec:
                    spec[param] = standard_params[param]
                else:
                    for sub_param in standard_params[param]:
                        spec[param][sub_param] = standard_params[param][sub_param]
            # Also add a blank line to the subtitle if one is not present
            if ensure_subtitle_space and "title" in spec:
                if "subtitle" not in spec["title"]:
                    spec["title"]["subtitle"] = ""
                elif type(spec["title"]["subtitle"]) != list:
                    spec["title"]["subtitle"] = [spec["title"]["subtitle"], ""]
                elif len(spec["title"]["subtitle"][-1]) > 1:
                    spec["title"]["subtitle"].append("")
            # Update descriptions
            if update_descriptions and "subtitle" in spec["title"] and "text" in spec["title"]:
                print( "A chart titled '" + spec["title"]["text"] + "' with subtitle '" + " ".join(spec["title"]["subtitle"]) + "'")
                spec["description"] = "A chart titled '" + spec["title"]["text"] + "' with subtitle '" + " ".join(spec["title"]["subtitle"])
            json.dump(spec, open(os.path.join(root, file), "w"), indent=2) # Overwrite the file


A chart titled 'US Incomes Over Time' with subtitle 'Real Household Incomes by Percentile, Race, and Region, over time (2021 USD) source: US Census Bureau '
A chart titled 'Household Income in South-West England' with subtitle 'Gross Domestic Household Income, Per Head (2020 Prices)  source: ONS '
A chart titled 'Britain's Beveridge Curve' with subtitle 'Job Vacancy Rate and Unemployment Rate by Month, Both Seasonally Adjusted Source: Bureau of Labour Statistics '
A chart titled 'Lifetime Earnings by Education' with subtitle ' Lifetime Earnings (Millions of USD) for Men and Women, with controls Estimates from Tamborini et al (2015) '
A chart titled 'US Bond Yields' with subtitle 'Par Yield Curve Rates Since 2007, Recessions Shaded Source: US Treasury '
A chart titled 'US Yield Curve' with subtitle '10 Year - 2 year Spread, Recessions Shaded Source: US Treasury '
A chart titled 'Chicago Area Median Incomes' with subtitle 'Median Individual Income, Workers Only   2021 Prices, source: Cen

In [24]:
" ".join(["hello", "world"])

'hello world'